In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import os

In [12]:
filepaths = ['..\Widerstand']

In [38]:
measurements = {}
for filepath in filepaths:
    for name in os.listdir(filepath):
        if name.endswith('.xlsx'):
            df = pd.read_excel(filepath + '\\' + name)
            arr = np.transpose(df.to_numpy())
            measurements[name.split('.')[0]] = arr
            print(arr.shape)


(7, 4160)
(7, 3035)
(7, 4113)
(7, 2887)
(7, 4499)
(7, 2889)


In [39]:
measurements['probe2_dw'][-1].shape

(4160,)

In [77]:
for name in measurements:
    time = measurements[name][-1]
    resistance = measurements[name][-2]
    voltage = measurements[name][-3]
    current = measurements[name][-4]
    temperature = measurements[name][0]

    fig,axs = plt.subplots(nrows=2,ncols=2,dpi=120)
    fig.tight_layout(pad=2.5, w_pad=3.5, h_pad=3.5)
    fig.suptitle(name)
    
    axs[0,0].scatter(time,temperature)
    axs[0,0].set(xlabel="time in s",ylabel="temperature in K")

    axs[0,1].scatter(time,current)
    axs[0,1].set(xlabel="time in s",ylabel="current in A")

    axs[1,0].scatter(time,voltage)
    axs[1,0].set(xlabel="time in s",ylabel="voltage in V")

    axs[1,1].scatter(time,resistance)
    axs[1,1].set(xlabel="time in s",ylabel="resistance in Ohm")

    ax = fig.add_subplot(110)
    ax.scatter(temperature,resistance)
    ax.set(xlabel="temperature in K",ylabel="resistance in Ohm")



ValueError: Number of columns must be a positive integer, not 0